In [4]:
pip install Bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 5.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [71]:
#name of a fasta file as input & return sequences as a list
from Bio import SeqIO
sequences = []
for seq_record in SeqIO.parse("/Users/sahanabaskar/Downloads/dna.example.fasta","fasta"):
    sequences.append(str(seq_record.seq))
    sequences.append(str(seq_record.id))
    print(sequences)

2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50


In [94]:
#total no of seq

sequences = []
for seq_record in SeqIO.parse("/Users/sahanabaskar/Downloads/dna2.fasta", "fasta"):
    sequences.append((str(seq_record.id), str(seq_record.seq)))

print(f"Total number of sequences: {len(sequences)}")


Total number of sequences: 18


In [93]:
#len of seq
sequences = []
for seq_record in SeqIO.parse("/Users/sahanabaskar/Downloads/dna2.fasta", "fasta"):
    sequences.append((str(seq_record.id),str(seq_record.seq)))
    print(len(seq_record.seq))

4635
1151
4894
3511
4076
2867
442
890
967
4338
1352
4564
4804
964
2095
1432
115
2646


In [95]:

sequences = []
for seq_record in SeqIO.parse("/Users/sahanabaskar/Downloads/dna2.fasta", "fasta"):
    seq_id = seq_record.id
    seq_len = len(seq_record.seq)
    sequences.append((seq_id, seq_len))

# Sort the sequences by length in descending order
sequences.sort(key=lambda x: x[1], reverse=True)

# Print the lengths of all sequences
print("Lengths of sequences:")
for seq_id, seq_len in sequences:
    print(f"{seq_id}\t{seq_len}")

# Print the longest sequence(s)
longest_seq_len = sequences[0][1]
longest_seqs = [(seq_id, seq_len) for seq_id, seq_len in sequences if seq_len == longest_seq_len]
print(f"\nLongest sequence(s) ({longest_seq_len} bp):")
for seq_id, seq_len in longest_seqs:
    print(seq_id)

# Print the shortest sequence(s)
shortest_seq_len = sequences[-1][1]
shortest_seqs = [(seq_id, seq_len) for seq_id, seq_len in sequences if seq_len == shortest_seq_len]
print(f"\nShortest sequence(s) ({shortest_seq_len} bp):")
for seq_id, seq_len in shortest_seqs:
    print(seq_id)
    
   

Lengths of sequences:
gi|142022655|gb|EQ086233.1|255	4894
gi|142022655|gb|EQ086233.1|16	4804
gi|142022655|gb|EQ086233.1|91	4635
gi|142022655|gb|EQ086233.1|454	4564
gi|142022655|gb|EQ086233.1|293	4338
gi|142022655|gb|EQ086233.1|396	4076
gi|142022655|gb|EQ086233.1|45	3511
gi|142022655|gb|EQ086233.1|250	2867
gi|142022655|gb|EQ086233.1|527	2646
gi|142022655|gb|EQ086233.1|4	2095
gi|142022655|gb|EQ086233.1|277	1432
gi|142022655|gb|EQ086233.1|75	1352
gi|142022655|gb|EQ086233.1|304	1151
gi|142022655|gb|EQ086233.1|594	967
gi|142022655|gb|EQ086233.1|584	964
gi|142022655|gb|EQ086233.1|88	890
gi|142022655|gb|EQ086233.1|322	442
gi|142022655|gb|EQ086233.1|346	115

Longest sequence(s) (4894 bp):
gi|142022655|gb|EQ086233.1|255

Shortest sequence(s) (115 bp):
gi|142022655|gb|EQ086233.1|346


In [107]:
#longest ORF

def find_orfs(sequence, frame):
    """Finds all ORFs in the given reading frame of a DNA sequence."""
    start_codon = "ATG"
    stop_codons = ["TAA", "TAG", "TGA"]
    orfs = []
    in_orf = False
    orf_start = 1
    
    for i in range(frame-1, len(sequence)-2, 3):
        codon = sequence[i:i+3]
        if codon == start_codon:
            # Start a new ORF
            in_orf = True
            orf_start = i
        elif codon in stop_codons:
            # End the current ORF
            if in_orf:
                orfs.append((orf_start, i+2))
                in_orf = False
                
    # Handle the case where the sequence ends in an ORF
    if in_orf:
        orfs.append((orf_start, len(sequence)-1))
        
    return orfs

# Read in the multi-FASTA file
records = list(SeqIO.parse("/Users/sahanabaskar/Downloads/dna2.fasta", "fasta"))

# Find all ORFs in each sequence in the forward reading frame
orf_data = []
for record in records:
    seq_id = record.id
    sequence = str(record.seq)
    orfs = find_orfs(sequence, frame=1)
    for orf_start, orf_stop in orfs:
        orf_len = orf_stop - orf_start + 1
        orf_data.append((seq_id, orf_start, orf_len))

# Determine the length of the longest ORF in the file and its location
longest_orf_len = max([data[2] for data in orf_data])
longest_orfs = [data for data in orf_data if data[2] == longest_orf_len]
longest_orf_locs = [(data[0], data[1]) for data in longest_orfs]
print(f"Length of longest ORF: {longest_orf_len} bp")
print("Identifiers of sequences containing longest ORF:")
for seq_id, orf_start in longest_orf_locs:
    print(f"{seq_id} (start position: {orf_start})")

# Determine the longest ORF for each sequence
max_orfs = []
for record in records:
    seq_id = record.id
    sequence = str(record.seq)
    orfs = find_orfs(sequence, frame=1)
    if orfs:
        max_orf_len = max([orf_stop-orf_start+1 for orf_start, orf_stop in orfs])
        max_orfs.append((seq_id, max_orf_len))
        
print("\nLongest ORF for each sequence:")
for seq_id, max_orf_len in max_orfs:
    print(f"{seq_id}: {max_orf_len} bp")


Length of longest ORF: 813 bp
Identifiers of sequences containing longest ORF:
gi|142022655|gb|EQ086233.1|396 (start position: 2373)

Longest ORF for each sequence:
gi|142022655|gb|EQ086233.1|91: 510 bp
gi|142022655|gb|EQ086233.1|304: 105 bp
gi|142022655|gb|EQ086233.1|255: 627 bp
gi|142022655|gb|EQ086233.1|45: 21 bp
gi|142022655|gb|EQ086233.1|396: 813 bp
gi|142022655|gb|EQ086233.1|250: 438 bp
gi|142022655|gb|EQ086233.1|322: 16 bp
gi|142022655|gb|EQ086233.1|88: 111 bp
gi|142022655|gb|EQ086233.1|594: 46 bp
gi|142022655|gb|EQ086233.1|293: 117 bp
gi|142022655|gb|EQ086233.1|75: 105 bp
gi|142022655|gb|EQ086233.1|454: 114 bp
gi|142022655|gb|EQ086233.1|16: 222 bp
gi|142022655|gb|EQ086233.1|584: 90 bp
gi|142022655|gb|EQ086233.1|4: 249 bp
gi|142022655|gb|EQ086233.1|277: 99 bp
gi|142022655|gb|EQ086233.1|346: 16 bp
gi|142022655|gb|EQ086233.1|527: 195 bp


In [105]:
#frequent repeats

def find_repeats(fasta_file, n):
    sequences = {}
    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        seq = str(seq_record.seq)
        for i in range(len(seq) - n + 1):
            repeat = seq[i:i+n]
            if repeat in sequences:
                sequences[repeat] += 1
            else:
                sequences[repeat] = 1
    max_count = 0
    max_repeat = ''
    for repeat in sequences:
        count = sequences[repeat]
        if count > max_count:
            max_count = count
            max_repeat = repeat
    print(f"Most frequent repeat of length {n}: {max_repeat} ({max_count} occurrences)")
    return sequences

fasta_file = "/Users/sahanabaskar/Downloads/dna2.fasta"
n = 7
repeats = find_repeats(fasta_file, n)


Most frequent repeat of length 7: CGCGCCG (63 occurrences)
